In [1]:
import pickle

# Replace 'file_path.pkl' with the actual path to your pickle file
file_path = '/home/tyler/Documents/Matt3r/imu-classification/DATA/train_8s_stage3/ambient_turn/0a0a0b1a-27d9fc44_3.pkl'

# Open the pickle file in read-binary mode ('rb')
with open(file_path, 'rb') as file:
    # Load the contents of the pickle file into a Python object
    data = pickle.load(file)

In [3]:
data.keys()

dict_keys(['locations', 'accelerometer', 'gyro', 'metadata', 'label', 'matt3r_format'])